# SIADS 593 WN26 Milestone I: Sustainable Water Quality Notebook

**Authors:**

Sungmin Kim  
Randy Best  
Kyle Rodriguez

#### **Introduction**

Welcome to the Exploratory Data Analysis of water quality across various river locations in South Africa! As a key factor for environmental sustainability, we would like to use data to identify and emphasize the key factors that significantly influence water quality. As a team we will take advantage of the valuable opportunity to apply data exploration techniques to real-world environmental data and contribute to advancing water quality monitoring.

**Primary Dataset**

With a dataset containing three target water quality parameters **Total Alkalinity**, **Electrical Conductance**, and **Dissolved Reactive Phosphorus** collected between 2011 and 2015 from approximately 200 river locations across South Africa, our goal is to find insight on which environmental conditions most affect these 3 metrics. Each data point includes the geographic coordinates (latitude and longitude) of the sampling site, the date of collection, and the corresponding water quality measurements. Features of the dataset also include four spectral bands—**SWIR22** (Shortwave Infrared 2), **NIR** (Near Infrared), **Green**, and **SWIR16** (Shortwave Infrared 1)—were utilized from Landsat, along with derived spectral indices such as **NDMI** (Normalized Difference Moisture Index) and **MNDWI** (Modified Normalized Difference Water Index). In addition, the **PET** (Potential Evapotranspiration) variable was incorporated from the **TerraClimate** dataset to account for climatic influences on water quality.

- **SWIR22** – Sensitive to surface moisture and turbidity variations in water bodies.  
- **NIR** – Helps in identifying vegetation and suspended matter in water.  
- **Green** – Useful for detecting water color and surface reflectance changes.  
- **SWIR16** – Provides information on surface dryness and sediment concentration.  
- **NDMI** – Derived from NIR and SWIR16, indicates moisture and vegetation–water interaction.  
- **MNDWI** – Derived from Green and SWIR22, effective for distinguishing open water areas and reducing built-up noise.  
- **PET** – Extracted from the TerraClimate dataset, represents potential evapotranspiration influencing hydrological and water quality dynamics.

The dataset spans a five-year period from 2011 to 2015. Using API-based data extraction methods, both Landsat and TerraClimate features were retrieved directly from the <a href="https://planetarycomputer.microsoft.com/">Microsoft Planetary Computer</a> portal. These combined spectral, index-based, and climatic features were used as predictors in a regression model to estimate three key water quality parameters: Total Alkalinity (TA), Electrical Conductance (EC), and Dissolved Reactive Phosphorus (DRP).

**Secondary Datasets**

Our secondary datasets comprise gridded population estimation from <a href="https://hub.worldpop.org/project/categories?id=18">WorldPop :: Population Density</a> and a <a href="https://en.wikipedia.org/wiki/List_of_rivers_of_South_Africa">List of rivers of South Africa</a> from Wikipedia. The population density data includes geographical coordinates and the number of people per square kilometre based on country totals adjusted to match the corresponding official United Nations population estimates that have been prepared by the Population Division of the Department of Economic and Social Affairs of the United Nations Secretariat (<a href="https://population.un.org/wpp/">2019 Revision of World Population Prospects</a>). The list of rivers dataset includes river name, province and location, source location and mouth/junction at their location name and geographical coordinates.

- Gridded population estimates are particularly useful as they provide decision-makers and data users with the flexibility to aggregate population estimates into different spatial units in existing enumeration areas or custom areas. Estimated population density per grid-cell. The projection is Geographic Coordinate System, WGS84. 
- The water sources of each quality sample in our original data are linked to geographical coordinates. So, we will use a simple list of rivers to reference each sample to its nominal water source.
 

 #### **About the Notebook**

In this notebook, we **load previously extracted data** from CSV files generated in a separate extraction notebook. This approach ensures a smoother and faster workflow, allowing participants to focus on data analysis and model development without waiting for time-consuming data retrieval.

## Load In Dependencies

To run this demonstration notebook, you will need to have the following packages imported below installed. This may take some time.  

```%pip install numpy pandas geopandas tqdm requests lxml matplotlib plotly seaborn```

In [1]:
# Data manipulation and analysis
import numpy as np
import pandas as pd
import geopandas as gpd

from datetime import date
from tqdm import tqdm
import os
import requests
from io import StringIO
import glob

# Visualization libraries
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns

# Suppress warnings
# import warnings
# warnings.filterwarnings('ignore')

%load_ext watermark
%watermark -v
%watermark --iversions

Python implementation: CPython
Python version       : 3.11.9
IPython version      : 9.9.0

geopandas : 1.1.2
matplotlib: 3.10.8
numpy     : 2.4.1
pandas    : 3.0.0
plotly    : 6.5.2
requests  : 2.32.5
seaborn   : 0.13.2
tqdm      : 4.67.1



## Load Data

In [2]:
# Water Quality Data
import primary_dataset

wq_data = primary_dataset.primary_dataset()

We will explore Water Quality over the course of 60 months.


In [3]:
# List of rivers of South Africa
# Wikipedia URL
url = "https://en.wikipedia.org/wiki/List_of_rivers_of_South_Africa"

# Custom headers to mimic a real browser
headers = {
    "User-Agent": "Chrome/107.0.0.0 Safari/537.36"
}

# Send HTTP request with headers
response = requests.get(url, headers=headers)

# Check if the request succeeded
if response.status_code == 200:
    # Read HTML tables from the response content
    tables = pd.read_html(StringIO(response.text))

else:
    print(f"Failed to fetch the page. Status code: {response.status_code}")

rivers = tables[1]
rivers.info()

<class 'pandas.DataFrame'>
RangeIndex: 292 entries, 0 to 291
Data columns (total 8 columns):
 #   Column                               Non-Null Count  Dtype
---  ------                               --------------  -----
 0   River                                292 non-null    str  
 1   Drainage basin[A]                    87 non-null     str  
 2   Province and location                291 non-null    str  
 3   Source location  (town / mountains)  79 non-null     str  
 4   Tributary of (river)                 79 non-null     str  
 5   Dam in river                         61 non-null     str  
 6   Mouth / junction at location (town)  64 non-null     str  
 7   Mouth / junction coordinates         57 non-null     str  
dtypes: str(8)
memory usage: 18.4 KB


In [4]:
#########################################################################
# Load Population Density data
# base_path = "/content/drive/My Drive/2026 EY Data Challenge/Dataset/Population_density"
# countries = gpd.read_file(
#     "/content/drive/My Drive/2026 EY Data Challenge/Dataset/ne_50m_admin_0_countries.shp"
# ).to_crs("EPSG:4326")
#########################################################################

<h3>Handling Missing Values</h3>  
<p align="justify">  
Before model training, missing values in the dataset were carefully handled to ensure data consistency and prevent model bias. Numerical columns were imputed using their median values, maintaining the overall data distribution while minimizing the impact of outliers.  
</p>

In [5]:
# wq_data = wq_data.fillna(wq_data.median(numeric_only=True))
wq_data.isna().sum()

Latitude                         0
Longitude                        0
Sample Date                      0
nir                              0
green                            0
swir16                           0
swir22                           0
NDMI                             0
MNDWI                            0
pet                              0
Total Alkalinity                 0
Electrical Conductance           0
Dissolved Reactive Phosphorus    0
binned_months                    0
dtype: int64

##################################################################################################################################################
##################################################################################################################################################

## Joining the datasets

In [ ]:

# 1) Copy original dataset
df = wq_data.copy()

# 2) Clean column names (safe + recommended)
df.columns = df.columns.str.strip().str.lower()

# 3) Create GeoDataFrame from lat/lon
gdf_points = gpd.GeoDataFrame(
    df,
    geometry=gpd.points_from_xy(df["longitude"], df["latitude"]),
    crs="EPSG:4326"
)
# #########################################################################
# 4) Load Natural Earth country boundaries
countries = gpd.read_file(
    # "/content/drive/My Drive/2026 EY Data Challenge/Dataset/ne_50m_admin_0_countries.shp"
    "ne_50m_admin_0_countries.shp"
).to_crs("EPSG:4326")
# #########################################################################

# 5) Spatial join to add country column
gdf_with_country = gpd.sjoin(
    gdf_points,
    countries[["ADMIN", "geometry"]],
    how="left",
    predicate="within"
)

# 6) Rename + cleanup
gdf_with_country = (
    gdf_with_country
    .rename(columns={"ADMIN": "country"})
    .drop(columns=["geometry", "index_right"], errors="ignore")
)

# 7) Move `country` to be the FIRST column
cols = gdf_with_country.columns.tolist()
cols.insert(0, cols.pop(cols.index("country")))
final_df = gdf_with_country[cols]

# 8) Quick check
final_df.head(100)

,country,latitude,longitude,sample date,total alkalinity,electrical conductance,dissolved reactive phosphorus,nir,green,swir16,swir22,ndmi,mndwi,pet
0,Namibia,-28.760833,17.730278,2011-01-02,128.912,555.0,10.0,11190.0,11426.0,7687.5,7645.0,0.185538,0.195595,174.20000
1,South Africa,-26.861111,28.884722,2011-01-03,74.720,162.9,163.0,17658.5,9550.0,13746.5,10574.0,0.124566,-0.180134,124.10000
2,South Africa,-26.450000,28.085833,2011-01-03,89.254,573.0,80.0,15210.0,10720.0,17974.0,14201.0,-0.083293,-0.252805,127.50000
3,South Africa,-27.671111,27.236944,2011-01-03,82.000,203.6,101.0,14887.0,10943.0,13522.0,11403.0,0.048048,-0.105416,129.70000
4,South Africa,-27.356667,27.286389,2011-01-03,56.100,145.1,151.0,16828.5,9502.5,12665.5,9643.0,0.141147,-0.142683,129.20000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,South Africa,-25.810483,27.909552,2011-01-26,110.000,652.0,10.0,16870.5,9753.0,13408.0,10257.0,0.114355,-0.157808,164.10000
96,South Africa,-34.065833,20.404167,2011-01-26,68.815,758.0,10.0,13535.0,8387.0,10165.0,8604.0,0.142194,-0.095839,179.30000
97,South Africa,-24.058889,31.237222,2011-01-26,70.928,306.4,15.0,14757.5,9468.5,13111.5,10821.0,0.059062,-0.161337,154.60000
98,South Africa,-26.648056,27.089444,2011-01-26,243.000,556.0,12.0,19310.5,9597.0,12954.5,9447.5,0.196994,-0.148881,170.90001


In [57]:
country_list = sorted(final_df["country"].dropna().unique())
country_list

['Lesotho', 'Mozambique', 'Namibia', 'South Africa']

In [ ]:
##################################################################
# Base directory
base_path = "/content/drive/My Drive/2026 EY Data Challenge/Dataset/Population_density"
##################################################################

# Map country codes to full names
country_map = {
    "zaf": "South Africa",
    "nam": "Namibia",
    "moz": "Mozambique",
    "lso": "Lesotho"
}

all_dfs = []

# Find all CSV files
csv_files = sorted(glob.glob(os.path.join(base_path, "*.csv")))

for file in csv_files:
    filename = os.path.basename(file)
    parts = filename.split("_")

    # Basic safety check on filename structure
    if len(parts) < 3:
        continue

    # Extract metadata from filename
    country_code = parts[0].lower()
    year = int(parts[2])
    country_name = country_map.get(country_code, "Unknown")

    # Read CSV
    df = pd.read_csv(file)

    # Rename XYZ columns to descriptive names
    df = df.rename(columns={
        "X": "longitude",
        "Y": "latitude",
        "Z": "population_density"
    })

    # Insert metadata columns
    df.insert(0, "country", country_name)
    df.insert(1, "year", year)

    all_dfs.append(df)

# Combine all countries & years into one DataFrame
population_density_all = pd.concat(all_dfs, ignore_index=True)

# Sort for consistency
population_density_all = population_density_all.sort_values(
    by=["country", "year", "latitude", "longitude"]
).reset_index(drop=True)

# Quick check
population_density_all.head()

,country,year,longitude,latitude,population_density
0,Lesotho,2011,28.111250,-30.654583,16.628876
1,Lesotho,2011,27.986250,-30.646250,15.893247
2,Lesotho,2011,27.994583,-30.646250,14.892983
3,Lesotho,2011,28.002917,-30.646250,15.519014
4,Lesotho,2011,28.011250,-30.646250,15.734591


In [59]:
population_density_all.isna().any()

,0
country,False
year,False
longitude,False
latitude,False
population_density,False


In [60]:
population_density_all.shape

(18337720, 5)

In [61]:
import numpy as np
import pandas as pd
from scipy.spatial import cKDTree

# --- 1) Copies + normalize column names ---
wq = final_df.copy()
pd_all = population_density_all.copy()

wq.columns = wq.columns.str.strip().str.lower()
pd_all.columns = pd_all.columns.str.strip().str.lower()

# --- 2) Ensure numeric types ---
for col in ["longitude", "latitude"]:
    wq[col] = pd.to_numeric(wq[col], errors="coerce")
    pd_all[col] = pd.to_numeric(pd_all[col], errors="coerce")

pd_all["population_density"] = pd.to_numeric(pd_all["population_density"], errors="coerce")
pd_all["year"] = pd.to_numeric(pd_all["year"], errors="coerce")

# --- 3) Extract sample year ---
wq["sample_year"] = pd.to_datetime(wq["sample date"], errors="coerce").dt.year

# --- 4) Initialize output columns ---
wq["pd_year"] = np.nan
wq["pop_density_nn"] = np.nan
wq["distance_km_to_pd_cell"] = np.nan

# --- 5) Helper: approx distance in km ---
def approx_deg_to_km(dlon, dlat, lat):
    km_lat = dlat * 111.0
    km_lon = dlon * 111.0 * np.cos(np.deg2rad(lat))
    return np.sqrt(km_lat**2 + km_lon**2)

# --- 6) Filter PD to clean rows only ---
pd_all_clean = pd_all.dropna(subset=["year", "longitude", "latitude", "population_density"]).copy()
pd_all_clean["year"] = pd_all_clean["year"].astype(int)

# --- 7) Build KDTree per year once ---
trees = {}
pd_by_year = {}

for yr, group in pd_all_clean.groupby("year"):
    group = group.reset_index(drop=True)
    coords = group[["longitude", "latitude"]].to_numpy()
    trees[yr] = cKDTree(coords)
    pd_by_year[yr] = group

# --- 8) Match WQ points year-by-year ---
valid_wq = wq.dropna(subset=["sample_year", "longitude", "latitude"]).copy()
valid_wq["sample_year"] = valid_wq["sample_year"].astype(int)

for yr in sorted(valid_wq["sample_year"].unique()):
    if yr not in trees:
        continue

    idx_rows = valid_wq.index[valid_wq["sample_year"] == yr]
    query_pts = valid_wq.loc[idx_rows, ["longitude", "latitude"]].to_numpy()

    dist, idx = trees[yr].query(query_pts, k=1)
    matched = pd_by_year[yr].iloc[idx].reset_index(drop=True)

    wq.loc[idx_rows, "pd_year"] = yr
    wq.loc[idx_rows, "pop_density_nn"] = matched["population_density"].to_numpy()

    dlon = valid_wq.loc[idx_rows, "longitude"].to_numpy() - matched["longitude"].to_numpy()
    dlat = valid_wq.loc[idx_rows, "latitude"].to_numpy() - matched["latitude"].to_numpy()
    lat  = valid_wq.loc[idx_rows, "latitude"].to_numpy()

    wq.loc[idx_rows, "distance_km_to_pd_cell"] = approx_deg_to_km(dlon, dlat, lat)

# --- 9) Build FINAL joined dataset: all original columns + new features ---
original_cols = final_df.copy()
original_cols.columns = original_cols.columns.str.strip().str.lower()
original_cols = original_cols.columns.tolist()

new_cols = ["pop_density_nn", "distance_km_to_pd_cell"]

# Keep original order + add new cols at the end (only if they exist)
final_cols = original_cols + [c for c in new_cols if c in wq.columns]
wq_joined = wq[final_cols].copy()

# --- 10) Show everything (not a subset) ---
wq_joined.head()

,country,latitude,longitude,sample date,total alkalinity,electrical conductance,dissolved reactive phosphorus,nir,green,swir16,swir22,ndmi,mndwi,pet,pop_density_nn,distance_km_to_pd_cell
0,Namibia,-28.760833,17.730278,2011-01-02,128.912,555.0,10.0,11190.0,11426.0,7687.5,7645.0,0.185538,0.195595,174.2,0.576990,0.325999
1,South Africa,-26.861111,28.884722,2011-01-03,74.720,162.9,163.0,17658.5,9550.0,13746.5,10574.0,0.124566,-0.180134,124.1,5.049022,0.251093
2,South Africa,-26.450000,28.085833,2011-01-03,89.254,573.0,80.0,15210.0,10720.0,17974.0,14201.0,-0.083293,-0.252805,127.5,23.239988,0.418343
3,South Africa,-27.671111,27.236944,2011-01-03,82.000,203.6,101.0,14887.0,10943.0,13522.0,11403.0,0.048048,-0.105416,129.7,687.465759,0.069949
4,South Africa,-27.356667,27.286389,2011-01-03,56.100,145.1,151.0,16828.5,9502.5,12665.5,9643.0,0.141147,-0.142683,129.2,6.092811,0.231730


In [62]:
wq_joined.isna().sum().sort_values(ascending=False)

,0
country,0
latitude,0
longitude,0
sample date,0
total alkalinity,0
electrical conductance,0
dissolved reactive phosphorus,0
nir,0
green,0
swir16,0


In [63]:
wq_joined.shape

(9319, 16)

In [64]:
wq_joined.head(100)

,country,latitude,longitude,sample date,total alkalinity,electrical conductance,dissolved reactive phosphorus,nir,green,swir16,swir22,ndmi,mndwi,pet,pop_density_nn,distance_km_to_pd_cell
0,Namibia,-28.760833,17.730278,2011-01-02,128.912,555.0,10.0,11190.0,11426.0,7687.5,7645.0,0.185538,0.195595,174.20000,0.576990,0.325999
1,South Africa,-26.861111,28.884722,2011-01-03,74.720,162.9,163.0,17658.5,9550.0,13746.5,10574.0,0.124566,-0.180134,124.10000,5.049022,0.251093
2,South Africa,-26.450000,28.085833,2011-01-03,89.254,573.0,80.0,15210.0,10720.0,17974.0,14201.0,-0.083293,-0.252805,127.50000,23.239988,0.418343
3,South Africa,-27.671111,27.236944,2011-01-03,82.000,203.6,101.0,14887.0,10943.0,13522.0,11403.0,0.048048,-0.105416,129.70000,687.465759,0.069949
4,South Africa,-27.356667,27.286389,2011-01-03,56.100,145.1,151.0,16828.5,9502.5,12665.5,9643.0,0.141147,-0.142683,129.20000,6.092811,0.231730
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,South Africa,-25.810483,27.909552,2011-01-26,110.000,652.0,10.0,16870.5,9753.0,13408.0,10257.0,0.114355,-0.157808,164.10000,41.214993,0.318925
96,South Africa,-34.065833,20.404167,2011-01-26,68.815,758.0,10.0,13535.0,8387.0,10165.0,8604.0,0.142194,-0.095839,179.30000,1.191061,0.343563
97,South Africa,-24.058889,31.237222,2011-01-26,70.928,306.4,15.0,14757.5,9468.5,13111.5,10821.0,0.059062,-0.161337,154.60000,1.350832,0.457766
98,South Africa,-26.648056,27.089444,2011-01-26,243.000,556.0,12.0,19310.5,9597.0,12954.5,9447.5,0.196994,-0.148881,170.90001,0.255399,0.374990


In [68]:
wq_joined["distance_km_to_pd_cell"].agg(
    ["min", "max", "mean", "median"]
)

,distance_km_to_pd_cell
min,0.025842
max,0.609135
mean,0.342514
median,0.358162


**How to interpret this**

You’re matching water-quality points to 1 km population-density grid cells.

Your results:
	•	Mean distance ≈ 0.34 km (340 meters)
	•	Median ≈ 0.36 km
	•	Max ≈ 0.61 km

For a 1 km grid, this is exactly what we want to see.

Why?

	•	In a 1 km × 1 km cell, the furthest possible distance from the center is ~0.71 km.
	•	Your max (0.61 km) is below that theoretical worst case.
	•	A mean around 0.35 km means most matches are happening well within the same grid cell or very close neighbors.

This tells us:

	•	Spatial alignment is tight
	•	Nearest-neighbor matching worked correctly
	•	No red flags like multi-km mismatches